## Tutorial 2: The log-likelihood function

The log-likelihood function is a mathematical function that represents the log of the probability of observing the given data under a specific statistical model, as a function of the model parameters. It is used to estimate the parameters of the choice model that are most likely to have produced the observed data.<br>

**This tutorial will shed light on the likelihood function**.<br> 
**It will cover the following topics :**
1. **What is the likelihood function and how we do we compute it**<br><br>
2. **How is the likelihood a function of the model parameters (i.e. the betas)**<br><br>
3. **Why do we usually work with the log-likelihood instead of the likelihood (and why this is permissable)**<br><br>
4. **What happens to the log-likelihood function when a model is misspecified (due to collinearity)**

In [1]:
# Import the libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import plotly.graph_objects as go
import biogeme.database as db
import biogeme.biogeme as bio
from biogeme import models
from biogeme.expressions import Beta, Variable, log

#### 1. What is the likelihood function and how we do we compute it?

Let's see how the likelihood of the data is calculated and how the likelihood changes as a function of the parameter values of a model.<br><br>
For this tutorial we use the data created in tutorial 1, and work with the linear-additive RUM-MNL model.

In [2]:
# Load the choice data
data_path =  Path(f'data/synthetic_VTTdata_tutorial1.dat')
df = pd.read_csv(data_path)

# Show the data
df.head(10)

,RESP,TC1,TT1,TC2,TT2,CHOICE
0,1,4,35,8,25,2
1,1,4,35,8,30,2
2,1,6,40,7,20,2
3,1,5,30,6,25,2
4,1,5,40,7,20,1
5,2,4,35,8,25,1
6,2,4,35,8,30,1
7,2,6,40,7,20,2
8,2,5,30,6,25,2
9,2,5,40,7,20,2


First, let's see how the likelihood of the data is calculated for a given set of betas.

In [3]:
# Set of betas
beta_tt = -0.2
beta_tc = -0.5

print('Set of betas')
print(f'beta_tt = {beta_tt}')
print(f'beta_tc = {beta_tc}')

Set of betas
beta_tt = -0.2
beta_tc = -0.5


To calculate the likelihood of the data given the linear-additive RUM-MNL model for a given set of betas, we need to calculate the probability of observing the data for each individual, and then multiply these probabilities together. The likelihood of the data is the product of the probabilities of observing the data for each individual.

In [4]:
# Compute the utilities given the set of betas 
df['V1'] = beta_tt * df['TT1'] + beta_tc *df['TC2']
df['V2'] = beta_tt * df['TT2'] + beta_tc *df['TC2']

# Compute the MNL choice probabilities using the logit formula
df['P1'] = np.exp(df['V1'])/(np.exp(df['V1']) + np.exp(df['V2']))
df['P2'] = np.exp(df['V2'])/(np.exp(df['V1']) + np.exp(df['V2']))

# Show the results
df.head()

,RESP,TC1,TT1,TC2,TT2,CHOICE,V1,V2,P1,P2
0,1,4,35,8,25,2,-11.0,-9.0,0.119203,0.880797
1,1,4,35,8,30,2,-11.0,-10.0,0.268941,0.731059
2,1,6,40,7,20,2,-11.5,-7.5,0.017986,0.982014
3,1,5,30,6,25,2,-9.0,-8.0,0.268941,0.731059
4,1,5,40,7,20,1,-11.5,-7.5,0.017986,0.982014


In [5]:
# Compute the likelihood of each observation given the model (RUM-MNL) and the set of betas
# For the likelihood only the probability of the chosen alternative is relevant
df['P_chosen'] = (df['CHOICE']==1) * df['P1'] + (df['CHOICE']==2) * df['P2']

df.head()

,RESP,TC1,TT1,TC2,TT2,CHOICE,V1,V2,P1,P2,P_chosen
0,1,4,35,8,25,2,-11.0,-9.0,0.119203,0.880797,0.880797
1,1,4,35,8,30,2,-11.0,-10.0,0.268941,0.731059,0.731059
2,1,6,40,7,20,2,-11.5,-7.5,0.017986,0.982014,0.982014
3,1,5,30,6,25,2,-9.0,-8.0,0.268941,0.731059,0.731059
4,1,5,40,7,20,1,-11.5,-7.5,0.017986,0.982014,0.017986


In [6]:
# The likelihood of the data given the model is the product of the likelihoods of the chosen alternatives
likelihood = df['P_chosen'].prod()
print(f'The likelihood of the data given the RUM-MNL model, with beta_tt = {beta_tt} and beta_tc = {beta_tc} is: {likelihood}')

The likelihood of the data given the RUM-MNL model, with beta_tt = -0.2 and beta_tc = -0.5 is: 1.297776207845552e-275


`--> Based on this calculation, we can make a couple of observations:` <br><br>
`1. The likelihood of the data given the model is increadibly small.`<br><br>
`2. Such small numbers can cause numerical issues. The smallest number computers can handle is 1e-308.`<br>
`In case we would have more observations (or a worse model), the likelihood would be smaller than the smallest number the computer can handle. If that happens a computer will churn our -inf, stop and prompt errors.`<br>
`Therefore, we usually work with the log of the likelihood to circumvent this problem.`

Let's see what happens when we calculate and work with the log of the likelihood.

In [7]:
# Compute the log-likelihood
df['logP_chosen'] = np.log(df['P_chosen'])

# Show the dataframe
df.head()

,RESP,TC1,TT1,TC2,TT2,CHOICE,V1,V2,P1,P2,P_chosen,logP_chosen
0,1,4,35,8,25,2,-11.0,-9.0,0.119203,0.880797,0.880797,-0.126928
1,1,4,35,8,30,2,-11.0,-10.0,0.268941,0.731059,0.731059,-0.313262
2,1,6,40,7,20,2,-11.5,-7.5,0.017986,0.982014,0.982014,-0.018150
3,1,5,30,6,25,2,-9.0,-8.0,0.268941,0.731059,0.731059,-0.313262
4,1,5,40,7,20,1,-11.5,-7.5,0.017986,0.982014,0.017986,-4.018150


In [8]:
# Compute the log-likelihood (sum of the log-likelihood of each observation)
log_likelihood = df['logP_chosen'].sum()
print(f'The log-likelihood of the data given the RUM-MNL model, with beta_tt = {beta_tt} and beta_tc = {beta_tc} is: {log_likelihood:0.3f}')

The log-likelihood of the data given the RUM-MNL model, with beta_tt = -0.2 and beta_tc = -0.5 is: -632.950


`--> Based on this calculation, we can make a couple of observations:` <br><br>
`1. The log-likelihood of the data given the model is properly scaled (not too small or large).`<br><br>
`2. The log-likelihood is negative, as the log of a number between 0 and 1 is always negative.`

### 2. How is the likelihood a function of the model parameters (i.e. the betas)?
In the previous cells, we computed the log-likelihood of the data for $\beta_{tt} = -0.2$ and $\beta_{tc} = -0.5$<br>
Next, we would like to see how the log-likelihood is a **function** of the model paremeters (i.e. the set of betas).
<br><br>
To visualise this, we first create a function that calculates the log-likelihood of the data for a given set of betas. We then use the fucntion to create a plot showing the log-likelihood as a function of the betas.

In [9]:
def compute_LL(beta_tt, beta_tc, df):
    
    # Compute the utilities given the set of betas
    V1 = beta_tt * df['TT1'] + beta_tc *df['TC1']
    V2 = beta_tt * df['TT2'] + beta_tc *df['TC2']

    # Compute the MNL choice probabilities using the logit formula
    P1 = np.exp(V1)/(np.exp(V1) + np.exp(V2))
    P2 = np.exp(V2)/(np.exp(V1) + np.exp(V2))

    # Compute the likelihood of each observation given the model (RUM-MNL) and betas
    P_chosen = (df['CHOICE']==1) * P1 + (df['CHOICE']==2) * P2

    # Compute the log-likelihood
    logP_chosen = np.log(P_chosen)

    # Compute the log-likelihood (sum of the log-likelihood of each observation)
    log_likelihood = np.sum(logP_chosen)
    return log_likelihood

Create the plot of the log-likelihood as a function of the betas using the `compute_LL` function.<br>
We use the `plotly` package to create an interactive plot.You can rotate the plot and zoom in and out to see the log-likelihood surface from different angles.

In [10]:
# Create meshgrid of betas
beta_tt = np.linspace(-0.16,-0.08  , 51)
beta_tc = np.linspace(-0.3,-0.1, 51)
BETA_TT, BETA_TC = np.meshgrid(beta_tt, beta_tc)

# Compute the log-likelihood for each pair of betas
LL = np.zeros_like(BETA_TT)
for i in range(BETA_TT.shape[0]):
    for j in range(BETA_TT.shape[1]):
        LL[i, j] = compute_LL(BETA_TT[i, j], BETA_TC[i, j], df)
        
# Plot the log-likelihood using plotly
vmin = LL.max() - 20
vmax = LL.max()
fig = go.Figure(data=[go.Surface(z = LL, x = BETA_TT, y = BETA_TC,cmin=vmin,cmax=vmax,colorscale='Viridis')])
fig.update_layout(title='Log-likelihood surface',
                  scene = dict(
                    xaxis_title='beta_tt',
                    yaxis_title='beta_tc',
                    zaxis_title='Log-likelihood'))

# Set z range
fig.update_layout(scene = dict(zaxis = dict(range=[vmin,vmax])))

# Increase the plot size
fig.update_layout(width=800, height=800)

# Show the plot
fig.show()

`--> We can make a couple of observation about the log-likelihood function:`<br><br>
`1. The log-likelihood function has a maximum around beta_tt = -0.126, beta_tc = -0.176. Any other combination of betas leads to a deterioration of the log-likelihood. Note that this is also what we found using Biogeme in tutorial 1 (see section 3. Bias)`<br><br>
`2. The log-likelihood function is globally concave. A globally concave function is like a hill: no matter where you are on the hill, the slope always goes downward from the peak. For linear-additive MNL models it is mathematically proven the log-likelood function is always globally concave.`<br>
`--> Because the log-likelihood function is globally concave for MNL models, the starting values are inconsequetial. After all, wherever you start the optimisation, if you keep climing up you will always end up in the same peak (which is the global maximum).`<br><br>
`3. In this case we can visually find the set of betas that maximise the (log-) likelihood of the data. But, in cause we would have 3 or more betas, it would not be possible to visualise the function`

### 3. Why do we usually work with the log-likelihood instead of the likelihood (and why this is permissable)?
The log is a monotonous transformation. Therefore, it should not affect the argument of the maximum of a function. I.e. <br>
$ \arg\max_{\beta}(\text{likelihood}) = \arg\max_{\beta}(\text{log(likelihood)})$ <br>
This can mathematically be proven. But we can also readily test it empirically using our data. To do so, we create the same plot as before, but now showing the likelihood on the z-axis.

 To show the how the likelihood is a function of the set of betas, we create a new function that calculates the **likelihood** of the data given a set of betas.

In [11]:
def compute_L(beta_tt, beta_tc, df):
    
    # Compute the utilities given the set of betas
    V1 = beta_tt * df['TT1'] + beta_tc *df['TC1']
    V2 = beta_tt * df['TT2'] + beta_tc *df['TC2']

    # Compute the MNL choice probabilities using the logit formula
    P1 = np.exp(V1)/(np.exp(V1) + np.exp(V2))
    P2 = np.exp(V2)/(np.exp(V1) + np.exp(V2))

    # Compute the likelihood of each observation given the model (RUM-MNL) and betas
    P_chosen = (df['CHOICE']==1) * P1 + (df['CHOICE']==2) * P2

    # Compute the likelihood
    likelihood = np.prod(P_chosen)

    return likelihood

Next, we create the plot of the likelihood as a function of the betas using the `compute_L` function.<br>

In [12]:
# Create meshgrid of betas
beta_tt = np.linspace(-0.16,-0.08  , 51)
beta_tc = np.linspace(-0.3,-0.1, 51)
BETA_TT, BETA_TC = np.meshgrid(beta_tt, beta_tc)

# Compute the log-likelihood for each pair of betas
L = np.zeros_like(BETA_TT)
for i in range(BETA_TT.shape[0]):
    for j in range(BETA_TT.shape[1]):
        L[i, j] = compute_L(BETA_TT[i, j], BETA_TC[i, j], df)
        
# Plot the log-likelihood using plotly
vmin = L.min()
vmax = L.max()
fig = go.Figure(data=[go.Surface(z = L, x = BETA_TT, y = BETA_TC,cmin=vmin,cmax=vmax,colorscale='Viridis')])
fig.update_layout(title='Likelihood surface',
                  scene = dict(
                    xaxis_title='beta_tt',
                    yaxis_title='beta_tc',
                    zaxis_title='Likelihood'))

# Set z range
fig.update_layout(scene = dict(zaxis = dict(range=[vmin,vmax])))

# Increase the plot size
fig.update_layout(width=800, height=800)

# Show the plot
fig.show()

`--> We can make a couple of observation about the likelihood function:`<br><br>
`1. The likelihood function has its maximum around beta_tt = -0.126, beta_tc = -0.176. Thus, indeed, taking the log does not affect the argument of the maximum.`<br><br>
`2. The z-axis is now in the positive domain. It ranges from 0 to 1.5e-229`


### 4. What happens to the log-likelihood function when a model is misspecified (due to collinearity)?
Occasionally, choice modellers try to estimate a discrete choice model that does not converge. This can happen due to several reasons, e.g.:<br>
1. Collinearity. In this case there are two attributes in the utility function that are one to one correlated. For instance, in revealed data distance and travel costare often highly correlated.
2. Misspecification. There may simply be a bug or logical error in the utility function
3. Complexity of the model. Sometimes a researcher wants to estimate such a complex model that it cannot be identified. In this case, the data is not sufficiently rich to be able to recover all parameters with sufficient precision.
A researcher may notice the poor convergence because the estimation algorithm tells her or the standard error are very large or suspicious.<br> 
In all of the above situations, are due to flatness of the log-likelihood function.

Let's see how the log-likelihood function looks like when the model is misspecified due to **collinearity**. First we create a collinear attribute: DIST. Then, we create a new function that calculates the log-likelihood of the data for a given set of betas, uisng the collinear attributes.

In [13]:
# Create a collinear attribute DIST = 10 + 0.05*TT
# DIST is collinear with TT
df['DIST1'] = 10 + 0.5 * df['TT1']
df['DIST2'] = 10 + 0.5 * df['TT2']

# Show the dataframe
df.head()

,RESP,TC1,TT1,TC2,TT2,CHOICE,V1,V2,P1,P2,P_chosen,logP_chosen,DIST1,DIST2
0,1,4,35,8,25,2,-11.0,-9.0,0.119203,0.880797,0.880797,-0.126928,27.5,22.5
1,1,4,35,8,30,2,-11.0,-10.0,0.268941,0.731059,0.731059,-0.313262,27.5,25.0
2,1,6,40,7,20,2,-11.5,-7.5,0.017986,0.982014,0.982014,-0.018150,30.0,20.0
3,1,5,30,6,25,2,-9.0,-8.0,0.268941,0.731059,0.731059,-0.313262,25.0,22.5
4,1,5,40,7,20,1,-11.5,-7.5,0.017986,0.982014,0.017986,-4.018150,30.0,20.0


We create the function to calculate the log-likelihood of the data for a given set of betas, using the collinear attributes.

In [14]:
def compute_LL_flat(beta_tt, beta_dist, df):
    
    # Compute the utilities given the set of betas
    # Note that we ignore the TC attribute so that we can still plot the log-likelihood function
    V1 = beta_tt * df['TT1'] + beta_dist *df['DIST1']
    V2 = beta_tt * df['TT2'] + beta_dist *df['DIST2']

    # Compute the MNL choice probabilities using the logit formula
    P1 = np.exp(V1)/(np.exp(V1) + np.exp(V2))
    P2 = np.exp(V2)/(np.exp(V1) + np.exp(V2))

    # Compute the likelihood of each observation given the model (RUM-MNL) and betas
    P_chosen = (df['CHOICE']==1) * P1 + (df['CHOICE']==2) * P2

    # Compute the log-likelihood
    logP_chosen = np.log(P_chosen)

    # Compute the log-likelihood (sum of the log-likelihood of each observation)
    log_likelihood = np.sum(logP_chosen)
    return log_likelihood

Finally, we plot the log-likelihood function.

In [15]:
# Create meshgrid of betas
beta_tt = np.linspace(-0.2,0  , 51)
beta_dist = np.linspace(-0.05,0.05, 51)
BETA_TT, BETA_DIST = np.meshgrid(beta_tt, beta_dist)

# Compute the log-likelihood for each pair of betas
LL = np.zeros_like(BETA_TT)
for i in range(BETA_TT.shape[0]):
    for j in range(BETA_TT.shape[1]):
        LL[i, j] = compute_LL_flat(BETA_TT[i, j], BETA_DIST[i, j], df)
        
# Plot the log-likelihood using plotly
vmin = LL.max() - 20
vmax = LL.max()
fig = go.Figure(data=[go.Surface(z = LL, x = BETA_TT, y = BETA_TC,cmin=vmin,cmax=vmax,colorscale='Viridis')])
fig.update_layout(title='Log-likelihood surface',
                  scene = dict(
                    xaxis_title='beta_tt',
                    yaxis_title='beta_dist',
                    zaxis_title='Log-likelihood'))

# Set z range
fig.update_layout(scene = dict(zaxis = dict(range=[vmin,vmax])))

# Increase the plot size
fig.update_layout(width=800, height=800)

# Show the plot
fig.show()

`--> We can make a couple of observation about the likelihood function:`<br><br>
`1. The log-likelihood function has a ridge. That is, the maximum is not a point, but a line.` <br><br>
`2. The maximum cannot be uniquely identified. By this we mean that we do not know arg max_beta (LL), unless we would fix one of the betas.`

Let's see what happens if we would estimate a choice model using TT and DIST using Biogeme.<br>
First, we create the database for biogeme. 

In [16]:
biodata = db.Database('synthetic_VTTdata',df)

# We create Variable objects for each of the variables in the data set that we want to use in the model
# Attributes of alternative 1
TT1  = Variable('TT1')
DIST1  = Variable('DIST1')

# Attributes of alternative 2    
TT2  = Variable('TT2')
DIST2  = Variable('DIST2')

# The choice
CHOICE = Variable('CHOICE')

Next, we define the model specification.

In [17]:
# Give a name to the model    
model_name = 'Linear-additive RUM-MNL model, using TT and DIST'

# Define the model parameters, using the function "Beta()":
B_TT = Beta('B_TT', 0, None, None, 0)
B_DIST = Beta('B_DIST', 0, None, None, 0)

# Define the utility functions
# Note there is no ASC in these utility functions
V1 = B_TT * TT1 + B_DIST * DIST1
V2 = B_TT * TT2 + B_DIST * DIST2

We create a function to estimate MNL models with two alternatives

In [18]:
# Create a function to estimate an MNL model with two alternatives
def estimate_mnl(V1,V2,CHOICE,database,model_name):

    V = {1: V1, 2: V2}
        
    # Create a dictionary called av to describe the availability conditions of each alternative, where 1 indicates that the alternative is available, and 0 indicates that the alternative is not available.
    # This shows that all alternatives were available to all respondents. 
    av = {1: 1, 2: 1} 

    # Define the choice model: The function models.logit() computes the MNL choice probabilities of the chosen alternative given the V. 
    prob = models.logit(V, av, CHOICE)

    # Define the log-likelihood   
    LL = log(prob)

    # Create the Biogeme object containing the object database and the formula for the contribution to the log-likelihood of each row using the following syntax:
    biogeme = bio.BIOGEME(database, LL)

    # The following syntax passes the name of the model:
    biogeme.modelName = model_name

    # Some object settings regaridng whether to save the results and outputs 
    biogeme.generate_pickle = False
    biogeme.generate_html = False
    biogeme.saveIterations = False

    # Syntax to calculate the null log-likelihood. The null-log-likelihood is used to compute the rho-square 
    biogeme.calculate_null_loglikelihood(av)

    # This line starts the estimation and returns the results object.
    results_MNL = biogeme.estimate()

    return results_MNL

Finally, we estimate the model and print the results.

In [19]:
# Estimate the model
results_MNL = estimate_mnl(V1,V2,CHOICE,biodata,model_name)

# Print the estimation statistics
print(results_MNL.short_summary())

# Print model parameters
print(results_MNL.get_estimated_parameters()) 

Results for model Linear-additive RUM-MNL model, using TT and DIST
Nbr of parameters:		2
Sample size:			1000
Excluded data:			0
Null log likelihood:		-693.1472
Final log likelihood:		-540.2048
Likelihood ratio test (null):		305.8848
Rho square (null):			0.221
Rho bar square (null):			0.218
Akaike Information Criterion:	1084.41
Bayesian Information Criterion:	1094.225

           Value   Rob. Std err    Rob. t-test  Rob. p-value
B_DIST -0.037845  1.797693e+308 -2.105214e-310           1.0
B_TT   -0.075691  1.797693e+308 -4.210428e-310           1.0


`--> We can make a couple of observations about the estimation results:`<br><br>
`1. At first sight, the estimation results look fine: we see a reasonable Log-likelihood and okay standard errors, etc ` <br><br>
`2. The fact that the t-values are EXACTLY equal is very suspicious, and suggest there may be a convergence issue.`<br><br>
`--> To further scrutinise the estimation results, we should inspect the correlations between the ESTIMATES in the estimation`

In [20]:
# We can see the correlations using  Biogeme's built-in function "getCorrelationResults":
results_MNL.get_correlation_results()

,Covariance,Correlation,t-test,p-value,Rob. cov.,Rob. corr.,Rob. t-test,Rob. p-value
B_TT-B_DIST,-2.228188e+10,-1.0,-1.195170e-07,1.0,1.806191e+09,1.797693e+308,1.797693e+308,0.0


`--> In the correlation results table we see that the correlation between B_TT and B_DIST is 1.0. Hence, in the estimates B_TT and B_DIST are one-to-one correlated`<br><br>
`--> For this, we can infer that the estimation has not converged. Threfore, the results cannot be trusted and should not be used`